In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using Theano backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [4]:
# Normalize the data
X_train = X_train / 255.0
test = test / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
test = test.values.reshape(-1,1,28,28)

In [5]:
Y_train = to_categorical(Y_train, num_classes = 10)

# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=42)
print(X_train.shape, Y_train.shape)

(37800, 1, 28, 28) (37800, 10)


In [6]:
model = Sequential()

# conv1
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same', 
                 activation ='relu', input_shape = (1,28,28)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))
# conv2
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# flatten
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))
# Compile the model

opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
print("compile done")

compile done


In [7]:
# With data augmentation to prevent overfitting (accuracy 0.99286)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

/home/jac/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (37800, 1, 28, 28) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [9]:
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.models import load_model

def lr_schedule(epoch):
    if epoch < 10:
        return 0.005
    elif epoch < 20:
        return 0.001
    elif epoch < 40:
        return 0.0005
    elif epoch < 60:
        return 0.0001
    else:
        return 0.00001

lr_s = LearningRateScheduler(lr_schedule)

best_model_path = 'best_m.h5'
best_model = ModelCheckpoint(filepath=best_model_path, monitor='val_acc', save_best_only=True)
epochs = 100
steps_per_epoch = 50
batch_size = 64
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=steps_per_epoch,
                              callbacks=[best_model, lr_s])
model = load_model(best_model_path)

Epoch 1/100
30s - loss: 0.2699 - acc: 0.9169 - val_loss: 0.1111 - val_acc: 0.9660
Epoch 2/100
29s - loss: 0.2880 - acc: 0.9109 - val_loss: 0.0844 - val_acc: 0.9757
Epoch 3/100
28s - loss: 0.2684 - acc: 0.9178 - val_loss: 0.0985 - val_acc: 0.9702
Epoch 4/100
30s - loss: 0.2384 - acc: 0.9287 - val_loss: 0.0788 - val_acc: 0.9762
Epoch 5/100
29s - loss: 0.2487 - acc: 0.9206 - val_loss: 0.0833 - val_acc: 0.9731
Epoch 6/100
30s - loss: 0.2331 - acc: 0.9319 - val_loss: 0.0630 - val_acc: 0.9793
Epoch 7/100
30s - loss: 0.2300 - acc: 0.9297 - val_loss: 0.0747 - val_acc: 0.9790
Epoch 8/100
30s - loss: 0.2008 - acc: 0.9375 - val_loss: 0.0600 - val_acc: 0.9824
Epoch 9/100
30s - loss: 0.1993 - acc: 0.9434 - val_loss: 0.0606 - val_acc: 0.9798
Epoch 10/100
31s - loss: 0.1709 - acc: 0.9472 - val_loss: 0.0701 - val_acc: 0.9793
Epoch 11/100
30s - loss: 0.1765 - acc: 0.9506 - val_loss: 0.0535 - val_acc: 0.9833
Epoch 12/100
31s - loss: 0.1782 - acc: 0.9466 - val_loss: 0.0542 - val_acc: 0.9831
Epoch 13/100


In [10]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [11]:
import arrow
ts = str(arrow.now())[:-10].replace(':','_').replace('.',"_")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("play_cnn_mnist_datagen_{}.csv".format(ts),index=False)

print('save done')

save done
